In [1]:
!pip install faiss-cpu sentence-transformers

     |████████████████████████████████| 8.0 MB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 483 kB 13.0 MB/s eta 0:00:01
     |████████████████████████████████| 11.6 MB 30.9 MB/s eta 0:00:01
     |████████████████████████████████| 3.1 MB 33.2 MB/s eta 0:00:01
     |████████████████████████████████| 288 kB 22.5 MB/s eta 0:00:01
     |████████████████████████████████| 563 kB 51.6 MB/s eta 0:00:01
     |████████████████████████████████| 2.8 MB 16.0 MB/s eta 0:00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.30.2
    Uninstalling huggingface-hub-0.30.2:
      Successfully uninstalled huggingface-hub-0.30.2


In [1]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from datetime import datetime

In [11]:
model = SentenceTransformer("all-MiniLM-L6-v2")

memory_texts = []
metadata_list = []

d = 384
index = faiss.IndexFlatL2(d)

In [10]:
def add_turn(text, players=[], npcs=[], tags=[], scene_id=None):
    embedding = model.encode([text], convert_to_numpy=True)
    index.add(embedding)
    memory_texts.append(text)
    metadata = {
        "text": text,
        "timestamp": datetime.utcnow().isoformat(),
        "players": players,
        "npcs": npcs,
        "tags": tags,
        "scene_id": scene_id
    }
    metadata_list.append(metadata)

In [7]:
def query_memory(query_text, top_k=3):
    query_embedding = model.encode([query_text], convert_to_numpy=True)
    D, I = index.search(query_embedding, top_k)

    results = []
    for rank, idx in enumerate(I[0]):
        result = {
            "text": memory_texts[idx],
            "distance": float(D[0][rank]),
            "metadata": metadata_list[idx]
        }
        results.append(result)
    return results


In [12]:
add_turn("Player1 opened the treasure chest", 
         players=["Player1"], 
         npcs=[], 
         tags=["exploration"], 
         scene_id=1)
add_turn("The party killed the dragon",
         players=["Player1","Player2","Player3"],
         npcs=["Dragon"],
         tags=["combat"],
         scene_id=2)
add_turn("Wizard asked to kill the bandits.",
         players=["Player1","Player2","Player3"],
         npcs=["Wizard"],
         tags=["quest"],
         scene_id=3)
add_turn("Bandits attacked the village",
         players=[],
         npcs=["Bandits"],
         tags=["dialogue"],
         scene_id=4)
add_turn("Player2 stole the key.",
         players=["Player2"],
         npcs=["Guard"],
         tags=["exploration"],
         scene_id=5)

In [13]:
query_results = query_memory("Who stole the key?", top_k=2)

for r in query_results:
    print(f"Match: {r['text']} | Distance={r['distance']:.2f} | Metadata={r['metadata']}")


Match: Player2 stole the key. | Distance=0.50 | Metadata={'text': 'Player2 stole the key.', 'timestamp': '2025-09-18T17:06:03.946574', 'players': ['Player2'], 'npcs': ['Guard'], 'tags': ['exploration'], 'scene_id': 5}
Match: Player1 opened the treasure chest | Distance=1.22 | Metadata={'text': 'Player1 opened the treasure chest', 'timestamp': '2025-09-18T17:06:03.891968', 'players': ['Player1'], 'npcs': [], 'tags': ['exploration'], 'scene_id': 1}
